### Imports

In [1]:
import os                                  #Hello2021
os.environ["OMP_NUM_THREADS"] = "8"        #"export" OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "8"   #"export" OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "8"        #"export" MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "8" #"export" VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "8"    #Hello2021

In [2]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
import implicit
import pickle
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype
from typing import List
import torch
import torch.nn.functional as F
from torch import nn
from polara.evaluation.pipelines import random_grid
import bottleneck as bn

In [3]:
import sys
sys.path.append('/home/oolaleke/incremental_PSI/Scripts/')
#sys.path.append('/content/')

In [4]:
import a_DataHelpers
import b_AlgoFunctions
import c_PredEval
import d_CorrScores
import e_VAEfunction

In [5]:
from a_DataHelpers import getPivotMonths,Time_DataSplit,TestTrain_DataSplit,SingleRatingMatrix
from a_DataHelpers import AllRatingMatrices,All_SingleStepRatMat,psiStep_RatMat,psiAllStep_RatMat
from a_DataHelpers import Find_NewUsersItems,adjustedPSI_DF,adjustedAllDF,get_NEWHoldout,ADJUST_mainDF
from a_DataHelpers import  SingleStepRatMat_2,AllSingleStepRatMat_2

from b_AlgoFunctions import integrator,getStartingValues,integratorOnMat,last_psiTrainMat, pureSVD
from b_AlgoFunctions import Updt_RowMatrix,getRow_Mat,row_update, Updt_ColMatrix,getCol_Mat,colunm_update
from b_AlgoFunctions import UsersItems_RatPair,getRowCol_psiupdt,ITEMS_defferredStatus,USERS_defferredStatus
from b_AlgoFunctions import getV_listUpdate, SingleStep_UPDATE,ALLSTEPs_UPDATE, auc_score,TrainTest
from b_AlgoFunctions import mean_auc,tune_ALS,nonzeros,least_squares_cg,alternating_least_squares_cg

from c_PredEval import TopNPred,TQDMgetALLTopNPred, Hitrate_Eval,getAll_HitRate,mean_confidence_interval
from c_PredEval import TQDMgetALLTopNPred_ALLUSERS,getALLTopNPred_ALLUSERS,get_ALLRandPred
from c_PredEval import topN_Index,getAll_RandomHitRate,getMOSTPOP_Pred,getAll_MostPOPHitRate,getAll_MOSTPOP_Pred
from c_PredEval import StepCoverage_Ratio,AllSteps_Coverage_Ratio,getAll_VAEMRR, MRR_Eval,getAll_MRR_Eval
from c_PredEval import TQDMgetAll_MRR_Eval, getAll_MostPOP_MRR,getAll_RandomRecMRR, ials_TopNPred,ials_getALLPred
from c_PredEval import ials_getALLPredTQDM,ials_ALLUSERSpred,ialsALLPred_ALLUSERS,TQDM_ialsALLPred_ALLUSERS

from d_CorrScores import no_copy_csr_matrix,build_rank_weights_matrix,rank_weighted_jaccard_index
from d_CorrScores import Updt_getAll_AvgCorr, updtCorr_4AllRanks

from e_VAEfunction import split_users, split_train_test, numerize_data, get_TrainRatMat, getTrainTest_RatMat


###iALS Functions

### Pred & Eval 

In [15]:
def topN_Index(a, n):
    parted = np.argpartition(a, -n)[-n:]
    return parted[np.argsort(-a[parted])]

def ials_TopNPred(RatingMat,holdout,users_vec,items_vec,user_column, N):  #N == Top_N
    TestUsers = holdout[user_column]            #prediction for holdout users alone
    HOLDOUT_usersMat = RatingMat[TestUsers,:]   #this doubles as the "previously seen items"
    testusers_vec = users_vec[TestUsers, :]
    PVVT =  testusers_vec.dot(items_vec.T) 
    users_column = HOLDOUT_usersMat.nonzero()[0]
    items_column = HOLDOUT_usersMat.nonzero()[1]
    args = np.array([users_column,items_column])
    np.put(PVVT, np.ravel_multi_index(args, PVVT.shape),-np.inf)   #downsample previously seen items
    TopN_pred = np.apply_along_axis(topN_Index, 1,PVVT,n = N)
    return TopN_pred

def ials_getALLPred(RatingMat_List,HOLDOUT_list,U_list,V_list,user_column,N):
    All_TOPN_PRED = []
    for RatMat,holdout,users_vec,items_vec in zip(RatingMat_List,HOLDOUT_list,U_list,V_list):
        TopN_pred =  ials_TopNPred(RatMat,holdout,users_vec,items_vec,user_column, N)  
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED

#RatingMat_List,HOLDOUT_list,U_list,V_list,
def ials_getALLPredTQDM(RatingMat_List,HOLDOUT_list,U_list,V_list,user_column,N):
    All_TOPN_PRED = []
    for RatMat,holdout,users_vec,items_vec in tqdm(zip(RatingMat_List,HOLDOUT_list,U_list,V_list)):  
        TopN_pred = ials_TopNPred(RatMat,holdout,users_vec,items_vec,user_column, N)
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED

def ials_ALLUSERSpred(RatingMat,users_vec,items_vec,N):  ##Prediction for all users ...||Not just Holdout
    PVVT =  users_vec.dot(items_vec.T) 
    users_column = RatingMat.nonzero()[0]
    items_column = RatingMat.nonzero()[1]
    args = np.array([users_column,items_column])
    np.put(PVVT, np.ravel_multi_index(args, PVVT.shape),-np.inf)   ##downsample previously seen items
    TopN_pred = np.apply_along_axis(topN_Index, 1,PVVT,n = N)
    return TopN_pred

def ialsALLPred_ALLUSERS(RatingMat_List,U_list,V_list,N):
    All_TOPN_PRED = []
    for RatMat,users_vec,items_vec in zip(RatingMat_List,U_list,V_list):  
        TopN_pred = ials_ALLUSERSpred(RatMat,users_vec,items_vec,N) 
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED

def TQDM_ialsALLPred_ALLUSERS(RatingMat_List,V_list,N):
    All_TOPN_PRED = []
    for RatMat,users_vec,items_vec in tqdm(zip(RatingMat_List,U_list,V_list)):  
        TopN_pred = ials_ALLUSERSpred(RatMat,users_vec,items_vec,N) 
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED

In [17]:
def getiALS_VUlist(bst_Model,UserItem_RatMAT,alpha):
    V_list = []
    U_list = []
    for RatMAT in tqdm(UserItem_RatMAT):
        Item_UsersMAT = (RatMAT.T*alpha).astype('double')
        bst_Model.fit(Item_UsersMAT, show_progress=True)                
        user_vecs = bst_Model.user_factors
        item_vecs = bst_Model.item_factors
        V_list.append(item_vecs)
        U_list.append(user_vecs)
    return V_list,U_list 

###MovieLen_DF

In [13]:
#MOVIElen_20M
newMLDF_20M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_afterUpdt.csv.zip')
newMLDF_20M['timestamp'] = pd.to_datetime(newMLDF_20M['timestamp']) 
print(newMLDF_20M.shape)   #userId_int
print(newMLDF_20M.dtypes)
newMLDF_20M.head()

(9990682, 9)
userid                     int64
movieId                    int64
rating                     int64
timestamp         datetime64[ns]
timestamp_YM              object
userId                     int64
productId                  int64
Updated_UserID             int64
Updated_ItemID             int64
dtype: object


,userid,movieId,rating,timestamp,timestamp_YM,userId,productId,Updated_UserID,Updated_ItemID
0,28507,1176,1,1995-01-09 11:46:44,1995-01,0,0,0,0
1,131160,47,1,1995-01-09 11:46:49,1995-01,1,1,1,1
2,85252,70,1,1996-01-29 00:00:00,1996-01,2,2,2,2
3,134445,45,1,1996-01-29 00:00:00,1996-01,3,3,3,3
4,99851,10,1,1996-01-29 00:00:00,1996-01,4,4,4,4


In [14]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newUserItem.pkl', 'rb') as f:
     newUserItem_list = pickle.load(f)


In [15]:
rows_i = newMLDF_20M['Updated_UserID'].nunique()
cols_i = newMLDF_20M['Updated_ItemID'].nunique()
UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

15


[<136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8865107 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8945455 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9025842 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9106051 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9186201 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9266468 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9346597 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9426837 stored elements in Compres

In [27]:
## latent factors= 10 ,regularization = 0.01:, n_iter = 30, alpha = 100,
rank=10
alpha=100
reglr=0.01
iter=30

In [28]:
best_model = implicit.als.AlternatingLeastSquares(factors=rank, regularization=reglr, iterations=iter,use_gpu = False)
best_model

In [30]:
V_list,U_list  = getiALS_VUlist(best_model,UserItem_MATList,alpha)
print(len(V_list),len(U_list))

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  7%|▋         | 1/15 [02:22<33:09, 142.08s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

 13%|█▎        | 2/15 [04:41<30:28, 140.66s/it]

...

 87%|████████▋ | 13/15 [32:05<05:08, 154.15s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

 93%|█████████▎| 14/15 [34:42<02:35, 155.03s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 15/15 [37:21<00:00, 149.44s/it]15 15



In [31]:
print(V_list[0].shape)
V_list[0]

(20720, 10)


array([[ 0.00138654, -0.00662851,  0.01164053, ..., -0.00381399,
        -0.010678  ,  0.03700994],
       [ 0.00406693,  0.0204256 , -0.0213815 , ..., -0.01375093,
         0.02026875,  0.02027158],
       [ 0.02341511, -0.00670435,  0.03356945, ..., -0.00883616,
         0.02762147,  0.01500012],
       ...,
       [ 0.00015684, -0.0007066 , -0.00023933, ...,  0.00029181,
         0.00024318,  0.00025619],
       [ 0.00015684, -0.0007066 , -0.00023933, ...,  0.00029181,
         0.00024318,  0.00025619],
       [ 0.00015684, -0.0007066 , -0.00023933, ...,  0.00029181,
         0.00024318,  0.00025619]], dtype=float32)

In [32]:
print(U_list[0].shape)
U_list[0] 

(136677, 10)


array([[ 0.42321783,  8.065394  , -1.5808942 , ...,  0.07281416,
         6.4723053 , 10.490261  ],
       [ 1.6069639 ,  5.6425533 , -1.7833235 , ..., -2.7991807 ,
         3.5406559 ,  2.6793363 ],
       [ 2.2058923 , 11.1626215 , -3.0424585 , ...,  0.9013025 ,
         8.036954  ,  7.0253887 ],
       ...,
       [-0.56736726, -1.4833525 , -2.1427412 , ..., -2.8038988 ,
         2.6899655 ,  3.239953  ],
       [-1.3589827 ,  1.2417485 , -0.49268597, ..., -3.454316  ,
         0.3824782 ,  6.3518124 ],
       [ 5.8199673 , -2.5476768 , -0.32295978, ..., -1.9410455 ,
         4.8684196 ,  9.782213  ]], dtype=float32)

In [33]:
top10_pred = ials_TopNPred(UserItem_MATList[0],newHoldout_list[0],U_list[0],V_list[0],'Updated_UserID', N=10)
print(top10_pred.shape)
top10_pred

(3459, 10)


array([[ 9142,  1565,  8982, ...,  8135, 15992,  8311],
       [  914,  2924,    62, ...,   406,  2996,  2714],
       [  146,  1027,  1310, ...,  1024,  6943,  7800],
       ...,
       [   78,    76,   149, ...,    20,  1062,   269],
       [ 1624,  9781,  2630, ...,  2378,  6760,  4724],
       [ 2008,  2547,  2235, ...,  3344,  2401,  2836]])

In [36]:
Hitrate_Eval(newHoldout_list[0],top10_pred,'Updated_UserID','Updated_ItemID')  #

Number of hits:  141
Total Num of users:  3459
Recommendation HitRate:  0.040763226366001735


0.040763226366001735

In [ ]:
##HitRate

In [37]:
All_ialsPRED = ials_getALLPredTQDM(UserItem_MATList,newHoldout_list,U_list,V_list,'Updated_UserID',N=10)
print(len(All_ialsPRED))


15it [00:08,  1.79it/s]15



In [38]:
AllSteps_Hitrate_iALS, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldout_list,All_ialsPRED,'Updated_UserID','Updated_ItemID')

Number of hits:  141
Total Num of users:  3459
Recommendation HitRate:  0.040763226366001735
Number of hits:  133
Total Num of users:  3311
Recommendation HitRate:  0.040169133192389
Number of hits:  128
Total Num of users:  3124
Recommendation HitRate:  0.040973111395646605
Number of hits:  137
Total Num of users:  3115
Recommendation HitRate:  0.043980738362760834
Number of hits:  137
Total Num of users:  3165
Recommendation HitRate:  0.04328593996840442
Number of hits:  111
Total Num of users:  3098
Recommendation HitRate:  0.035829567462879276
Number of hits:  118
Total Num of users:  3169
Recommendation HitRate:  0.0372357210476491
Number of hits:  144
Total Num of users:  3129
Recommendation HitRate:  0.046021093000958774
Number of hits:  142
Total Num of users:  3192
Recommendation HitRate:  0.044486215538847115
Number of hits:  157
Total Num of users:  3409
Recommendation HitRate:  0.04605456145497213
Number of hits:  127
Total Num of users:  3267
Recommendation HitRate:  0.038

In [39]:
AllSteps_Hitrate_iALS

[0.040763226366001735,
 0.040169133192389,
 0.040973111395646605,
 0.043980738362760834,
 0.04328593996840442,
 0.035829567462879276,
 0.0372357210476491,
 0.046021093000958774,
 0.044486215538847115,
 0.04605456145497213,
 0.03887358432812978,
 0.04127396815079623,
 0.045454545454545456,
 0.0397125567322239,
 0.034432234432234435]

In [40]:
##All prediction:
All_alsPRED_List = ialsALLPred_ALLUSERS(UserItem_MATList,U_list,V_list,N=10)  
print(len(All_alsPRED_List))
All_alsPRED_List[:2]

15


[array([[ 140,   17,  151, ...,   76,  108,   15],
        [  42,   19,   50, ...,  149,   58,  110],
        [ 304,  124, 1255, ...,  108,  195,   22],
        ...,
        [  78,  149,  105, ...,  861, 1062,   58],
        [4724, 4750, 2770, ..., 2378, 9781, 4137],
        [  78,   76,  149, ...,   92, 9781,   14]]),
 array([[ 140,   17,  151, ...,   76,  108,   15],
        [  42,   19,   50, ...,  149,   58,  110],
        [ 304,  124, 1255, ...,  108,  195,   22],
        ...,
        [  78,  149,  105, ...,  861, 1062,   58],
        [4724, 4750, 2770, ..., 2378, 9781, 4137],
        [  78,   76,  149, ...,   92, 9781,   14]])]

In [41]:
All_alsPRED_List[0].shape

(136677, 10)

In [45]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_ialsRT_allPRED.pkl', 'wb') as f:
     pickle.dump(All_alsPRED_List, f)


In [44]:
##mrr
#als_AllStepsHitR_, als_LBand, als_AvgHitR, als_UBand 
als_AllStepsMRR, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldout_list,All_ialsPRED,'Updated_ItemID')
als_AllStepsMRR    

15it [00:00, 504.51it/s]


[0.015236,
 0.013069,
 0.013784,
 0.015161,
 0.013493,
 0.010798,
 0.011916,
 0.013365,
 0.016248,
 0.014866,
 0.0151,
 0.011921,
 0.016177,
 0.01458,
 0.011589]

In [46]:
##Coverage
###cOVERAGE:
als_AvgCovList = AllSteps_Coverage_Ratio(newMLDF_20M,All_alsPRED_List,'Updated_ItemID')
als_AvgCovList

[0.118147,
 0.118098,
 0.118147,
 0.118243,
 0.118195,
 0.11834,
 0.118436,
 0.118533,
 0.118678,
 0.118871,
 0.118967,
 0.118822,
 0.119015,
 0.11916,
 0.119353]

In [48]:
AllSteps = list(range(1,15))
print(len(AllSteps))
AllSteps

14


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [49]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(All_alsPRED_List,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_

array([array([1.        , 1.        , 1.        , ..., 0.91813929, 1.        ,
              1.        ])                                                    ,
       array([1., 1., 1., ..., 1., 1., 1.]),
       array([1., 1., 1., ..., 1., 1., 1.]),
       array([1.        , 1.        , 1.        , ..., 0.69056344, 0.6699095 ,
              0.08321104])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.31733   , 0.91813929,
              0.70363531])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.56625995, 1.        ,
              0.90698876])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.7956453 , 1.        ,
              0.87216233])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.70836709, 0.91813929,
              0.09526636])          

In [50]:
UpdtPSICorr_.shape
for df in UpdtPSICorr_:
    print(df.mean())  

0.9987890598388818
0.999111298344693
0.9989668104384218
0.9990022883774632
0.9990717473639525
0.9990228969406075
0.9991534365690912
0.9992754676882192
0.9990608570015678
0.9989781160470768
0.9991394551052235
0.9988572673328814
0.998916032129617
0.999253391753015


###Amz_Beauty

In [84]:
#AMZBty
newAMZB_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_afterUpdt.csv')

In [85]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newUserItem.pkl', 'rb') as f:
     newUserItem_list = pickle.load(f)

In [86]:
rows_i = newAMZB_DF['Updated_UserID'].nunique()
cols_i = newAMZB_DF['Updated_ItemID'].nunique()
UserItem_MATList_amb = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList_amb))   
UserItem_MATList_amb

10


[<22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 40228 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 41330 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 42432 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 43525 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 44665 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 45752 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 47034 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 48112 stored elements in Compressed Sparse Row format>,
 <22557x

In [89]:
#latent factors= 10 , regularization = 1, n_iter = 15, alpha = 5, AUC = 0.79839
rank = 10
alpha = 5
reglr = 1
iter = 15
best_model_bty = implicit.als.AlternatingLeastSquares(factors=rank, regularization=reglr, iterations=iter,use_gpu = False)
best_model_bty

In [90]:
V_list_bty,U_list_bty  = getiALS_VUlist(best_model_bty,UserItem_MATList_amb,alpha)
print(len(V_list_bty),len(U_list_bty))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:05,  1.57it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:01<00:04,  1.72it/s]

...

 80%|████████  | 8/10 [00:04<00:01,  1.71it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [00:05<00:00,  1.71it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:06<00:00,  1.64it/s]10 10



In [91]:
All_ialsPRED_bty = ials_getALLPredTQDM(UserItem_MATList_amb,newHoldout_list,U_list_bty,V_list_bty,'Updated_UserID',N=10) 
print(len(All_ialsPRED_bty))

10it [00:00, 14.25it/s]10



In [92]:
AllStepsHitr_iALS_bty, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldout_list,All_ialsPRED_bty,'Updated_UserID','Updated_ItemID')
AllStepsHitr_iALS_bty

Number of hits:  148
Total Num of users:  474
Recommendation HitRate:  0.31223628691983124
Number of hits:  170
Total Num of users:  433
Recommendation HitRate:  0.39260969976905313
Number of hits:  186
Total Num of users:  465
Recommendation HitRate:  0.4
Number of hits:  182
Total Num of users:  478
Recommendation HitRate:  0.3807531380753138
Number of hits:  173
Total Num of users:  478
Recommendation HitRate:  0.3619246861924686
Number of hits:  201
Total Num of users:  508
Recommendation HitRate:  0.3956692913385827
Number of hits:  422
Total Num of users:  612
Recommendation HitRate:  0.6895424836601307
Number of hits:  252
Total Num of users:  599
Recommendation HitRate:  0.42070116861435725
Number of hits:  234
Total Num of users:  536
Recommendation HitRate:  0.43656716417910446
Number of hits:  104
Total Num of users:  425
Recommendation HitRate:  0.2447058823529412
Average HitRate for All Recommendations:  0.40347098011017835


[0.31223628691983124,
 0.39260969976905313,
 0.4,
 0.3807531380753138,
 0.3619246861924686,
 0.3956692913385827,
 0.6895424836601307,
 0.42070116861435725,
 0.43656716417910446,
 0.2447058823529412]

In [93]:
AllStepsHitr_iALS_bty

[0.31223628691983124,
 0.39260969976905313,
 0.4,
 0.3807531380753138,
 0.3619246861924686,
 0.3956692913385827,
 0.6895424836601307,
 0.42070116861435725,
 0.43656716417910446,
 0.2447058823529412]

In [96]:
##All prediction:
All_usersPRED_amb  = ialsALLPred_ALLUSERS(UserItem_MATList_amb,U_list_bty,V_list_bty,N=10)  
print(len(All_usersPRED_amb))
All_usersPRED_amb[:2]


10


[array([[ 168,  311,  842, ...,  555,  385,  228],
        [ 168,  311, 1230, ...,  555,  110,  385],
        [ 168,  311, 1230, ...,  321,  194,  322],
        ...,
        [ 692,  842, 2849, ...,  995,  385,  228],
        [2849,  995,   22, ...,  800,  557, 3666],
        [1230,  995, 2849, ...,  676,  231, 2120]]),
 array([[ 168,  311,  842, ...,  555,  385,  228],
        [ 168,  311, 1230, ...,  555,  110,  385],
        [ 168,  311, 1230, ...,  321,  194,  322],
        ...,
        [ 692,  842, 2849, ...,  995,  385,  228],
        [2849,  995,   22, ...,  800,  557, 3666],
        [1230,  995, 2849, ...,  676,  231, 2120]])]

In [98]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_ialsRT_allPRED.pkl', 'wb') as f:  
     pickle.dump(All_usersPRED_amb, f)

In [99]:
#als_AllStepsHitR_, als_LBand, als_AvgHitR, als_UBand 
als_AllStepsMRR_bty, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldout_list,All_ialsPRED_bty,'Updated_ItemID')
als_AllStepsMRR_bty

10it [00:00, 683.16it/s]


[0.28652,
 0.348428,
 0.344253,
 0.335785,
 0.313232,
 0.308541,
 0.54902,
 0.345979,
 0.365187,
 0.169874]

In [100]:
##Coverage
als_AvgCovList_bty = AllSteps_Coverage_Ratio(newAMZB_DF,All_usersPRED_amb,'Updated_ItemID')
als_AvgCovList_bty

[0.013812,
 0.013812,
 0.014182,
 0.014182,
 0.014182,
 0.014182,
 0.014428,
 0.014428,
 0.014428,
 0.014428]

In [101]:
AllSteps_10 = list(range(1,10))
print(len(AllSteps_10))
AllSteps_10

9


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [102]:
UpdtPSICorr_bty = Updt_getAll_AvgCorr(All_usersPRED_amb,newUserItem_list,AllSteps_10,'Updated_UserID')  #,
UpdtPSICorr_bty

array([array([1.        , 1.        , 1.        , ..., 0.23006416, 0.7956453 ,
              0.7956453 ])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.49096051, 1.        ,
              0.49096051])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.20752556, 0.49096051,
              0.70836709])                                                    ,
       array([1.        , 1.        , 1.        , ..., 1.        , 0.49096051,
              0.49096051])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.49096051, 1.        ,
              1.        ])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.49096051, 0.49096051,
              0.49096051])                                                    ,
       array([1.        , 1.        , 1.      

In [103]:

for df in UpdtPSICorr_bty:
    print(df.mean())  


0.9962956082015595
0.9958661801268679
0.9958300778352793
0.9958945613668483
0.9961923378849067
0.9944196210325439
0.9916817067157313
0.9948657173245663
0.9953391747756014


###MovieLen_1M

In [7]:
newMLDF_1M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_afterUpdt.csv.zip')
print(newMLDF_1M.shape)   #userId_int


(575281, 9)


In [8]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newHoldout.pkl', 'rb') as f:
     newHoldout_listML1 = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newUserItem.pkl', 'rb') as f:
     newUserItem_listML1 = pickle.load(f)

In [10]:
rows_i = newMLDF_1M['Updated_UserID'].nunique()
cols_i = newMLDF_1M['Updated_ItemID'].nunique()
UserItem_MATListML1 = AllRatingMatrices(newUserItem_listML1,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATListML1))   
UserItem_MATListML1

10


[<6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 500332 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 508812 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 517278 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 525757 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 534098 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 542538 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 551048 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 559597 stored elements in Compressed Sparse Row format>,
 <6038x3

In [12]:
## latent factors= 20 , regularization = 0.05, n_iter = 20, alpha = 50
#Ulist_als,Vlist_als = alternating_least_squares_cg(UserItem_RatMAT[-1],nuser_list,nItem_list,factors=20,alpha=50,reglr=0.05,iter=20)
rank = 20
reglr = 0.05
iter = 20
alpha=50

In [13]:
best_model_ml1 = implicit.als.AlternatingLeastSquares(factors=rank, regularization=reglr, iterations=iter,use_gpu = False)
best_model_ml1


In [18]:
V_list,U_list  = getiALS_VUlist(best_model_ml1,UserItem_MATListML1,alpha)
print(len(V_list),len(U_list))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:06<00:58,  6.46s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:13<00:52,  6.54s/it]

...

 80%|████████  | 8/10 [01:05<00:15,  7.89s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [01:11<00:07,  7.50s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:22<00:00,  8.28s/it]10 10



In [19]:
All_ialsPRED1m = ials_getALLPredTQDM(UserItem_MATListML1,newHoldout_listML1,U_list,V_list,'Updated_UserID',N=10)
print(len(All_ialsPRED1m))

10it [00:00, 28.89it/s]10



In [20]:
AllSteps_Hitrate_iALS1m, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldout_listML1,All_ialsPRED1m,'Updated_UserID','Updated_ItemID')

Number of hits:  13
Total Num of users:  233
Recommendation HitRate:  0.055793991416309016
Number of hits:  12
Total Num of users:  255
Recommendation HitRate:  0.047058823529411764
Number of hits:  29
Total Num of users:  286
Recommendation HitRate:  0.10139860139860139
Number of hits:  20
Total Num of users:  309
Recommendation HitRate:  0.06472491909385113
Number of hits:  18
Total Num of users:  468
Recommendation HitRate:  0.038461538461538464
Number of hits:  22
Total Num of users:  526
Recommendation HitRate:  0.04182509505703422
Number of hits:  18
Total Num of users:  517
Recommendation HitRate:  0.03481624758220503
Number of hits:  15
Total Num of users:  470
Recommendation HitRate:  0.031914893617021274
Number of hits:  16
Total Num of users:  453
Recommendation HitRate:  0.03532008830022075
Number of hits:  19
Total Num of users:  362
Recommendation HitRate:  0.052486187845303865
Average HitRate for All Recommendations:  0.05038003863014969


[0.055793991416309016,
 0.047058823529411764,
 0.10139860139860139,
 0.06472491909385113,
 0.038461538461538464,
 0.04182509505703422,
 0.03481624758220503,
 0.031914893617021274,
 0.03532008830022075,
 0.052486187845303865]

In [21]:
AllSteps_Hitrate_iALS1m

[0.055793991416309016,
 0.047058823529411764,
 0.10139860139860139,
 0.06472491909385113,
 0.038461538461538464,
 0.04182509505703422,
 0.03481624758220503,
 0.031914893617021274,
 0.03532008830022075,
 0.052486187845303865]

In [22]:
#All prediction:
All_usersialsPRED1m  = ialsALLPred_ALLUSERS(UserItem_MATListML1,U_list,V_list,N=10)  
print(len(All_usersialsPRED1m))
All_usersialsPRED1m[:2]

10


[array([[ 256,  320,  566, ...,  247, 1226,  279],
        [ 369,  563,  564, ...,  635, 1116,  140],
        [  79,  478,  695, ...,   23,  171,  641],
        ...,
        [ 472,  680,  975, ...,  669,   54, 1020],
        [  61,  921,  925, ...,   83,  574,  432],
        [  82,  897,   31, ...,  388,  905,  153]]),
 array([[ 256,  320,  566, ...,  247, 1226,  279],
        [ 369,  563,  564, ...,  635, 1116,  140],
        [  79,  478,  695, ...,   23,  171,  641],
        ...,
        [ 472,  680,  975, ...,  669,   54, 1020],
        [  61,  921,  925, ...,   83,  574,  432],
        [  82,  897,   31, ...,  388,  905,  153]])]

In [24]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/ML1_ialsRT_allPRED.pkl', 'wb') as f:
     pickle.dump(All_usersialsPRED1m, f)

In [25]:
#als_AllStepsHitR_, als_LBand, als_AvgHitR, als_UBand 
als_AllStepsMRR_ML1, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldout_listML1,All_ialsPRED1m,'Updated_ItemID')
als_AllStepsMRR_ML1

10it [00:00, 653.42it/s]


[0.012285,
 0.016704,
 0.038849,
 0.017435,
 0.007294,
 0.011021,
 0.010794,
 0.010074,
 0.00939,
 0.018622]

In [26]:
##Coverage
###cOVERAGE:
als_AvgCovList_ML1 = AllSteps_Coverage_Ratio(newMLDF_1M,All_usersialsPRED1m,'Updated_ItemID')
als_AvgCovList_ML1

[0.459949,
 0.459949,
 0.460515,
 0.461081,
 0.461081,
 0.461364,
 0.46193,
 0.461647,
 0.461364,
 0.461081]

In [27]:
AllSteps_10 = list(range(1,10))
print(len(AllSteps_10))
AllSteps_10

9


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [28]:
UpdtPSICorr_ML1 = Updt_getAll_AvgCorr(All_usersialsPRED1m,newUserItem_listML1,AllSteps_10,'Updated_UserID')  #,
UpdtPSICorr_ML1

array([array([1., 1., 1., ..., 1., 1., 1.]),
       array([1., 1., 1., ..., 1., 1., 1.]),
       array([1.        , 1.        , 1.        , ..., 0.87216233, 0.10040999,
              0.90698876])                                                    ,
       array([1.        , 1.        , 1.        , ..., 1.        , 1.        ,
              0.59227699])                                                    ,
       array([1., 1., 1., ..., 1., 1., 1.]),
       array([1., 1., 1., ..., 1., 1., 1.]),
       array([0.84271627, 1.        , 1.        , ..., 1.        , 0.87216233,
              0.65215445])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.49096051, 0.69463896,
              1.        ])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.17065821, 1.        ,
              1.        ])                                                    ],
      dtype=object)

In [29]:

for df in UpdtPSICorr_ML1:
    print(df.mean())  

0.9971061071900807
0.9970551355717188
0.9949711023697712
0.9917415400020714
0.9909368311232252
0.9917716375598122
0.9915170481352636
0.9931394334231475
0.9940270238412618


###Amz_Games

In [51]:
newAMZGames_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_afterUpdt.csv')

print(newAMZGames_DF.shape)   #userId_int
print(newAMZGames_DF.dtypes)
newAMZGames_DF.head()

(345203, 9)
userid            object
ProductID         object
rating             int64
timestamp         object
timestamp_YM      object
userId             int64
productId          int64
Updated_UserID     int64
Updated_ItemID     int64
dtype: object


,userid,ProductID,rating,timestamp,timestamp_YM,userId,productId,Updated_UserID,Updated_ItemID
0,A2AXQTB83VMK4L,B0000296O5,1,1999-10-14,1999-10,0,0,0,0
1,A2T04VAIXSKJH2,B00002NDRY,1,1999-11-05,1999-11,1,1,1,1
2,A1QA8K3LD9K892,B000021Y5F,1,1999-11-10,1999-11,2,2,2,2
3,A1QA8K3LD9K892,B000021Y5Q,1,1999-11-10,1999-11,2,3,2,3
4,ANLA598UNJI8A,B00001IVGE,1,1999-11-14,1999-11,3,4,3,4


In [54]:
##lOAD Saved Lists...
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newUserItem.pkl', 'rb') as f:   #
     newUserItem_list = pickle.load(f)


with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newHoldout.pkl', 'rb') as f:
     newHOLDOUT_LIST = pickle.load(f)


In [ ]:
#latent factors= 10 , regularization = 1, n_iter = 15, alpha = 5, AUC = 0.79839
Ulist_als,Vlist_als = alternating_least_squares_cg(UserItem_RatMAT[-1],nuser_list,nItem_list,factors=10,alpha=5,reglr=1,iter=15)
print(len(Ulist_als))  
Ulist_als[:2]

In [55]:
rows_i = newAMZGames_DF['Updated_UserID'].nunique()
cols_i = newAMZGames_DF['Updated_ItemID'].nunique()
UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

15


[<39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 250384 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 255520 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 261105 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 266597 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 272102 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 277572 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 282899 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 288493 stored elements in Compressed Sparse Row f

In [64]:
#est model; latent factors= 20 , regularization = 10, n_iter = 20, alpha = 50, AUC = 0.87067
#Ulist_als,Vlist_als = alternating_least_squares_cg(UserItem_RatMAT[-1],nuser_list,nItem_list,factors=20,alpha=50,reglr=10,iter=20)
rank = 20
alpha = 50
reglr = 10
iter = 20

In [65]:
best_model_gm = implicit.als.AlternatingLeastSquares(factors=rank, regularization=reglr, iterations=iter,use_gpu = False)
best_model_gm


In [66]:
V_list,U_list  = getiALS_VUlist(best_model_gm,UserItem_MATList,alpha)
print(len(V_list),len(U_list))

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  7%|▋         | 1/15 [00:04<01:05,  4.70s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

 13%|█▎        | 2/15 [00:09<01:05,  5.03s/it]

...

 87%|████████▋ | 13/15 [00:54<00:08,  4.40s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

 93%|█████████▎| 14/15 [00:58<00:04,  4.16s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 15/15 [01:01<00:00,  4.13s/it]15 15



In [67]:
print(V_list[0].shape)
V_list[0]

(17318, 20)


array([[ 2.4586728e-01, -1.0253001e-01,  5.2688974e-01, ...,
         9.5730409e-02,  5.0278598e-01, -2.1666187e-01],
       [-3.7283552e-01, -3.0116895e-01,  3.0186471e-01, ...,
        -2.8866857e-01,  3.0461004e-01,  3.3960751e-01],
       [-2.9005392e-02,  1.1430346e-02,  1.2192819e-02, ...,
         1.4158544e-02,  3.7464689e-02,  4.1886050e-02],
       ...,
       [ 9.9292891e-03, -8.2680453e-03, -7.2541721e-03, ...,
         5.0837616e-03,  3.9920737e-03, -7.1032578e-03],
       [-6.6471118e-03, -2.4426004e-04,  9.5815230e-03, ...,
         6.2665213e-03, -3.2993790e-03, -8.1900088e-03],
       [-2.0265041e-03, -2.3011619e-02,  3.4033705e-03, ...,
        -4.7611753e-03,  7.4624126e-03, -1.9644326e-02]], dtype=float32)

In [68]:
print(U_list[0].shape)
U_list[0] 

(39284, 20)


array([[-0.10047837,  0.00785078,  0.061002  , ...,  0.00421529,
         0.10790339,  0.0123235 ],
       [-0.00781376, -0.08831523,  0.12809968, ..., -0.0732992 ,
         0.04521151,  0.0849192 ],
       [-0.02811822, -0.00351027,  0.08058146, ...,  0.08152273,
         0.0970396 ,  0.02830829],
       ...,
       [ 0.01630092,  0.02622255,  0.1127287 , ...,  0.04876202,
         0.03764512, -0.01158503],
       [ 0.06569523,  0.10956392,  0.03666268, ..., -0.0239969 ,
         0.08159462, -0.03336352],
       [ 0.00759129, -0.00352367, -0.00453922, ...,  0.03935907,
         0.01827624, -0.02539729]], dtype=float32)

In [69]:
All_ialsPRED = ials_getALLPredTQDM(UserItem_MATList,newHOLDOUT_LIST,U_list,V_list,'Updated_UserID',N=10)
print(len(All_ialsPRED))

15it [00:08,  1.86it/s]15



In [73]:
All_ialsPRED[0].shape

(2018, 10)

In [75]:
AllSteps_Hitrate_iALS, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHOLDOUT_LIST,All_ialsPRED,'Updated_UserID','Updated_ItemID')

Number of hits:  274
Total Num of users:  2018
Recommendation HitRate:  0.13577799801783944
Number of hits:  303
Total Num of users:  2200
Recommendation HitRate:  0.13772727272727273
Number of hits:  374
Total Num of users:  2172
Recommendation HitRate:  0.1721915285451197
Number of hits:  319
Total Num of users:  2079
Recommendation HitRate:  0.15343915343915343
Number of hits:  286
Total Num of users:  2089
Recommendation HitRate:  0.13690761129727141
Number of hits:  264
Total Num of users:  2120
Recommendation HitRate:  0.12452830188679245
Number of hits:  271
Total Num of users:  2217
Recommendation HitRate:  0.12223725755525484
Number of hits:  310
Total Num of users:  2207
Recommendation HitRate:  0.14046216583597643
Number of hits:  311
Total Num of users:  2133
Recommendation HitRate:  0.14580403187998126
Number of hits:  371
Total Num of users:  2171
Recommendation HitRate:  0.1708889912482727
Number of hits:  378
Total Num of users:  2261
Recommendation HitRate:  0.16718266

In [76]:
AllSteps_Hitrate_iALS

[0.13577799801783944,
 0.13772727272727273,
 0.1721915285451197,
 0.15343915343915343,
 0.13690761129727141,
 0.12452830188679245,
 0.12223725755525484,
 0.14046216583597643,
 0.14580403187998126,
 0.1708889912482727,
 0.16718266253869968,
 0.1460373998219056,
 0.1343351548269581,
 0.1375055090348171,
 0.03915391539153915]

In [77]:
##All prediction:
All_usersialsPRED  = ialsALLPred_ALLUSERS(UserItem_MATList,U_list,V_list,N=10)  
print(len(All_usersialsPRED))
All_usersialsPRED[:2]

15


[array([[ 1050,  1434,   145, ...,  1552,   882,   610],
        [15455, 13773, 15563, ..., 11891, 17122, 16799],
        [ 1050,   385,   145, ...,   202,   754,   346],
        ...,
        [ 4255,  6027,  6055, ...,  7613,  7666,  6113],
        [13045, 13519, 15194, ..., 15487, 15562, 14806],
        [ 8077,  8019, 10344, ...,  7527,  7319,  4957]]),
 array([[ 1050,  1434,   145, ...,  1552,   882,   610],
        [15455, 13773, 15563, ..., 11891, 17122, 16799],
        [ 1050,   385,   145, ...,   202,   754,   346],
        ...,
        [ 4255,  6027,  6055, ...,  7613,  7666,  6113],
        [13045, 13519, 15194, ..., 15487, 15562, 14806],
        [ 8077,  8019, 10344, ...,  7527,  7319,  4957]])]

In [82]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZG_ialsRT_allPRED.pkl', 'wb') as f: 
     pickle.dump(All_usersialsPRED, f)

In [78]:
##mrr
#als_AllStepsHitR_, als_LBand, als_AvgHitR, als_UBand 
als_AllStepsMRR_azmg, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHOLDOUT_LIST,All_ialsPRED,'Updated_ItemID')
als_AllStepsMRR_azmg

15it [00:00, 578.59it/s]


[0.04772,
 0.053833,
 0.073051,
 0.061129,
 0.058823,
 0.053896,
 0.051082,
 0.058526,
 0.062536,
 0.076733,
 0.073479,
 0.061514,
 0.057076,
 0.064686,
 0.014476]

In [79]:
##Coverage
###cOVERAGE:
als_AvgCovList_amg = AllSteps_Coverage_Ratio(newAMZGames_DF,All_usersialsPRED,'Updated_ItemID')
als_AvgCovList_amg

[0.067444,
 0.067618,
 0.067618,
 0.06756,
 0.067675,
 0.067675,
 0.067675,
 0.067733,
 0.067791,
 0.067733,
 0.067675,
 0.067675,
 0.067618,
 0.067618,
 0.067618]

In [80]:
UpdtPSICorr_amg = Updt_getAll_AvgCorr(All_usersialsPRED,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_amg


array([array([1.       , 1.       , 1.       , ..., 1.       , 1.       ,
              0.7956453])                                                ,
       array([1.        , 1.        , 1.        , ..., 1.        , 0.49096051,
              1.        ])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.92690249, 1.        ,
              1.        ])                                                    ,
       array([1.        , 1.        , 1.        , ..., 0.93397092, 1.        ,
              0.70836709])                                                    ,
       array([1.        , 1.        , 1.        , ..., 1.        , 1.        ,
              0.84271627])                                                    ,
       array([1.       , 1.       , 1.       , ..., 0.7956453, 1.       ,
              1.       ])                                                ,
       array([1.        , 1.        , 1.        , ..., 1.        ,

In [81]:

for df in UpdtPSICorr_amg:
    print(df.mean())  

0.996910090121146
0.9966430925405705
0.9963338303261913
0.9966207847652867
0.9965970104139847
0.9970158089092586
0.9968901113543907
0.996415681302881
0.9965009204286434
0.9959621672464949
0.9959822219615734
0.9967086735077657
0.9965829631948259
0.996751554591299
